In [239]:
"""
"""


import os
import sys

import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

sys.path.append('F:/time_step/OfflineRL_FactoredActions')
from RL_mimic_sepsis.utils.timestep_util import get_horizon

In [240]:
timestep = 8
# Select model: 'create' or 'load'.
mode = 'normal'

save_dir = rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\data_asNormThreshold_dt{timestep}h'
os.makedirs(os.path.join(save_dir, f'episodes/'), exist_ok=True)
train_file = f'episodes/train_set_asNormThreshold_dt{timestep}h.pt'
val_file = f'episodes/val_set_asNormThreshold_dt{timestep}h.pt'
test_file = f'episodes/test_set_asNormThreshold_dt{timestep}h.pt'

# Define the features of the full data.
device = 'cpu'
num_actions = 25
num_obs = 33
num_dem = 5
num_acuity_scores = 3
horizon = get_horizon(timestep)        
action_temp = torch.eye(num_actions)

In [241]:
full_data_file = os.path.join(save_dir, f'sepsis_final_data_FILLED_withTimes_dt{timestep}h_asNormThreshold.csv')
acuity_scores_file = os.path.join(save_dir, f'acuity_scores_dt{timestep}h.csv')

full_zs = pd.read_csv(full_data_file)
acuity_scores = pd.read_csv(acuity_scores_file)

In [242]:
# Number of icustays before dumping.
len(full_zs['traj'].unique())

18783

In [243]:
# Load shared icustayids across all time step-sizes.
shared_ids_path = rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\shared_icustayids_plus_mask.csv'
shared_ids_df = pd.read_csv(shared_ids_path)
full_zs['m:icustayid']   = full_zs['m:icustayid'].astype('int64')
shared_ids_df['m:icustayid'] = shared_ids_df['m:icustayid'].astype('int64')

mask = full_zs['m:icustayid'].isin(shared_ids_df['m:icustayid'])
full_zs = full_zs[mask].copy()

In [244]:
len(full_zs['traj'].unique())

18377

In [245]:
if mode == 'create':
    # Determine the train, val, test split (70/15/15), stratified by patient outcome.
    temp = full_zs.groupby('traj')['r:reward'].sum()
    y = temp.values
    X = temp.index.values
    # Stratify ensures both 2 sets keep the same proportion of classes as in y.
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=np.array(y), test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.5, random_state=42)

    # Create the train, val, test sets from 1-h result.
    np.save(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\X_train.npy', X_train)
    np.save(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\X_val.npy', X_val)
    np.save(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\X_test.npy', X_test)
    np.save(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\y_train.npy', y_train)
    np.save(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\y_val.npy', y_val)
    np.save(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\y_test.npy', y_test)


In [246]:
if mode == 'normal':
    # Determine the train, val, test split (70/15/15), stratified by patient outcome.
    temp = full_zs.groupby('m:icustayid')['r:reward'].sum()
    y = temp.values
    X = temp.index.values
    # Stratify ensures both 2 sets keep the same proportion of classes as in y.
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=np.array(y), test_size=0.3, random_state=42)
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.5, random_state=42)


In [247]:
if mode == 'load':
    X_train = np.load(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\X_train.npy')
    X_val = np.load(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\X_val.npy')
    X_test = np.load(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\X_test.npy')
    y_train = np.load(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\y_train.npy')
    y_val = np.load(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\y_val.npy')
    y_test = np.load(rf'F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\split\y_test.npy')

In [248]:
# Check the head of the training set.
# It should be array([242850, 295268, 216505, 284889, 291455, 239924, 285353, 226420, 200451, 251991], dtype=int64)
X_train[:10]


array([242850, 295268, 216505, 284889, 291455, 239924, 285353, 226420,
       200451, 251991], dtype=int64)

In [249]:
X_train.shape

(12863,)

In [250]:
# Drop unneeded meta features.
full_zs = full_zs.drop(['m:presumed_onset', 'm:charttime'], axis=1)
full_zs['m:icustayid'] = full_zs['m:icustayid'].astype(int)

# 'train_data' is a pandas dataframe.
# train_data = full_zs[full_zs['traj'].isin(X_train)].copy()
train_data = full_zs[full_zs['m:icustayid'].isin(X_train)].copy()

In [251]:
len(train_data['m:icustayid'].unique())

12863

In [252]:
acuity_scores

,traj,m:icustayid,step,c:SOFA,c:OASIS,c:SAPSii
0,1,200003.0,0,5.0,11.0,15.0
1,1,200003.0,1,5.0,15.0,15.0
2,1,200003.0,2,5.0,11.0,15.0
3,1,200003.0,3,6.0,17.0,22.0
4,2,200011.0,0,3.0,30.0,47.0
...,...,...,...,...,...,...
132033,18783,299995.0,2,1.0,14.0,7.0
132034,18783,299995.0,3,0.0,8.0,3.0
132035,18783,299995.0,4,0.0,7.0,0.0
132036,18783,299995.0,5,0.0,12.0,4.0


In [253]:
# train_acuity = acuity_scores[acuity_scores['traj'].isin(X_train)]
train_acuity = acuity_scores[acuity_scores['m:icustayid'].isin(X_train)]
train_trajectories = train_data['traj'].unique()

In [254]:
train_trajectories.shape

(12863,)

In [255]:
# val_data = full_zs[full_zs['traj'].isin(X_val)]
# val_acuity = acuity_scores[acuity_scores['traj'].isin(X_val)]
val_data = full_zs[full_zs['m:icustayid'].isin(X_val)]
val_acuity = acuity_scores[acuity_scores['m:icustayid'].isin(X_val)]
val_trajectories = val_data['traj'].unique()

# test_data = full_zs[full_zs['traj'].isin(X_test)]
# test_acuity = acuity_scores[acuity_scores['traj'].isin(X_test)]
test_data = full_zs[full_zs['m:icustayid'].isin(X_test)]
test_acuity = acuity_scores[acuity_scores['m:icustayid'].isin(X_test)]
test_trajectories = test_data['traj'].unique()

In [256]:
full_zs['step'].max()

9

In [257]:
full_zs.head()

,traj,step,m:icustayid,o:gender,o:mechvent,o:re_admission,o:age,o:Weight_kg,o:GCS,o:HR,...,o:output_4hourly,r:reward,o:max_dose_vaso,o:input_4hourly,a:iv_OLD,a:vaso_OLD,a:vaso,a:iv,a:action_OLD,a:action
0,1,0,200003,-0.5,-0.5,-0.5,-0.967320,-0.179834,0.673587,-0.924091,...,0.614054,0.0,0.0,90.0,2,0,0,1,10,5
1,1,1,200003,-0.5,-0.5,-0.5,-0.967320,-0.161734,0.673587,-0.768957,...,0.505675,0.0,0.0,90.0,2,0,0,1,10,5
2,1,2,200003,-0.5,-0.5,-0.5,-0.967320,-0.139109,0.673587,-0.603480,...,0.589999,0.0,0.0,60.0,1,0,0,1,5,5
3,1,3,200003,-0.5,-0.5,-0.5,-0.967320,-0.139109,0.673587,0.215628,...,-2.029683,1.0,0.0,0.0,0,0,0,0,0,0
4,2,0,200011,0.5,0.5,0.5,1.183748,0.744639,0.673587,-0.752409,...,0.442832,0.0,0.0,0.0,0,0,0,0,0,0


In [258]:
full_zs[['o:input_total', 'o:input_4hourly', 'o:max_dose_vaso']].head()

,o:input_total,o:input_4hourly,o:max_dose_vaso
0,0.779339,90.0,0.0
1,0.788697,90.0,0.0
2,0.794866,60.0,0.0
3,0.794866,0.0,0.0
4,-6.846875,0.0,0.0


In [259]:
print('Total count:', len(np.unique(full_zs['m:icustayid'])))
print('Train:', len(y_train), y_train.mean())
print('Val  :', len(y_val), y_val.mean())
print('Test :', len(y_test), y_test.mean())

Total count: 18377
Train: 12863 0.8815206405970614
Val  : 2757 0.8810301051867973
Test : 2757 0.8817555313746827


In [260]:
################################################################
#          FORMAT DATA FOR USE IN SEQUENTIAL MODELS
################################################################

icustayid_col = 'm:icustayid'
# 5-D demographic informations.
dem_keep_cols = ['o:gender', 'o:mechvent', 'o:re_admission', 'o:age', 'o:Weight_kg']
# 33-D lab and vitals.
obs_keep_cols = ['o:GCS', 'o:HR', 'o:SysBP',
       'o:MeanBP', 'o:DiaBP', 'o:RR', 'o:Temp_C', 'o:FiO2_1', 'o:Potassium',
       'o:Sodium', 'o:Chloride', 'o:Glucose', 'o:Magnesium', 'o:Calcium',
       'o:Hb', 'o:WBC_count', 'o:Platelets_count', 'o:PTT', 'o:PT',
       'o:Arterial_pH', 'o:paO2', 'o:paCO2', 'o:Arterial_BE', 'o:HCO3',
       'o:Arterial_lactate','o:PaO2_FiO2', 'o:SpO2', 'o:BUN', 'o:Creatinine',
       'o:SGOT', 'o:SGPT', 'o:Total_bili', 'o:INR']

dem_cols = [i for i in train_data.columns if i in dem_keep_cols]
obs_cols = [i for i in train_data.columns if i in obs_keep_cols]
# Action columns.
ac_cols  = [i for i in train_data.columns if i[:2] == 'a:']
# Reward columns
rew_cols = [i for i in train_data.columns if i[:2] == 'r:']
acuity_cols = [i for i in train_acuity.columns if i[:2] == 'c:']


In [261]:
# Assuming discrete actions and scalar rewards:
assert len(obs_cols) > 0, 'No observations present, or observation columns not prefixed with "o:"'
assert len(ac_cols) > 0, 'No actions present, or actions column not prefixed with "a:"'
assert len(rew_cols) > 0, 'No rewards present, or rewards column not prefixed with "r:"'
assert len(rew_cols) == 1, 'Multiple reward columns are present when a single reward column is expected'
assert len(acuity_cols) == num_acuity_scores, 'Ensure that we have the right number of acuity scores'

ac_col = ac_cols[0]
rew_col = rew_cols[0]

## TRAINING DATA

In [262]:
df_data = train_data
df_acuity = train_acuity
trajectories = train_trajectories

In [263]:
trajectories.shape

(12863,)

In [264]:
icustayids = np.zeros((len(trajectories)), dtype=int)
lengths = np.zeros((len(trajectories)), dtype=int)
times = np.zeros((len(trajectories), horizon), dtype=int)
notdones = np.zeros((len(trajectories), horizon), dtype=int)
observations = np.zeros((len(trajectories), horizon, num_obs))
demographics = np.zeros((len(trajectories), horizon, num_dem))
acuities = np.zeros((len(trajectories), horizon, num_acuity_scores))
rewards = np.zeros((len(trajectories), horizon))
actions = np.zeros((len(trajectories), horizon))
actionvecs = np.zeros((len(trajectories), horizon, num_actions))
subactions = np.zeros((len(trajectories), horizon, 2))
subactionvecs = np.zeros((len(trajectories), horizon, 5+5))

for i, traj_idx in tqdm(enumerate(trajectories), total=len(trajectories)):
    traj = df_data[df_data['traj'] == traj_idx].sort_values(by='step')
    traj_acuity = df_acuity[df_acuity['traj'] == traj_idx].sort_values(by='step')
    length = traj.shape[0]
    assert len(traj_acuity == length)
    
    icustayids[i] = traj[icustayid_col].iloc[0]
    lengths[i] = length
    times[i] = np.arange(horizon)
    times[i, length:] = 0
    notdones[i, :length-1] = 1
    observations[i, :length, :] = traj[obs_cols].values
    demographics[i, :length, :] = traj[dem_cols].values
    acuities[i, :length, :] = traj_acuity[acuity_cols].values
    rewards[i, :length] = traj[rew_col].values
    actions[i, :length] = traj['a:action'].values
    # TODO: What is 'actionvecs'?
    actionvecs[i, range(length), traj['a:action'].astype(int).values] = 1
    subactions[i, :length, :] = traj[['a:vaso', 'a:iv']].values
    subactionvecs[i, range(length), traj['a:vaso'].astype(int).values] = 1
    subactionvecs[i, range(length), 5+traj['a:iv'].astype(int).values] = 1

100%|██████████| 12863/12863 [00:21<00:00, 602.48it/s]


In [265]:
observations.shape

(12863, 11, 33)

In [266]:
# Eliminate single transition trajectories.
length_mask = (lengths > 1.0)
trajectories = trajectories[length_mask]
icustayids = icustayids[length_mask]
lengths = lengths[length_mask]
times = times[length_mask]
notdones = notdones[length_mask]
observations = observations[length_mask]
demographics = demographics[length_mask]
acuities = acuities[length_mask]
rewards = rewards[length_mask]
actions = actions[length_mask]
subactions = subactions[length_mask]
actionvecs = actionvecs[length_mask]
subactionvecs = subactionvecs[length_mask]

In [267]:
observations.shape

(12863, 11, 33)

In [268]:
print(os.path.join(save_dir, train_file))

torch.save({
    'index': torch.Tensor(trajectories).long(),
    'icustayids': torch.Tensor(icustayids).long(),
    'lengths': torch.Tensor(lengths).long(), 
    'times': torch.Tensor(times).float(), 
    'notdones': torch.Tensor(notdones).long(), 
    'demographics': torch.Tensor(demographics).float(), 
    'observations': torch.Tensor(observations).float(), 
    'acuities': torch.Tensor(acuities).float(),
    'rewards': torch.Tensor(rewards).float(),
    'actions': torch.Tensor(actions).long(), 
    'actionvecs': torch.Tensor(actionvecs).float(), 
    'subactions': torch.Tensor(subactions).long(), 
    'subactionvecs': torch.Tensor(subactionvecs).float(), 
}, os.path.join(save_dir, train_file))

F:\time_step\OfflineRL_FactoredActions\RL_mimic_sepsis\data\data_asNormThreshold_dt8h\episodes/train_set_asNormThreshold_dt8h.pt


## Validation DATA

In [269]:
df_data = val_data
df_acuity = val_acuity
trajectories = val_trajectories

In [270]:
icustayids = np.zeros((len(trajectories)), dtype=int)
lengths = np.zeros((len(trajectories)), dtype=int)
times = np.zeros((len(trajectories), horizon), dtype=int)
notdones = np.zeros((len(trajectories), horizon), dtype=int)
observations = np.zeros((len(trajectories), horizon, num_obs))
demographics = np.zeros((len(trajectories), horizon, num_dem))
acuities = np.zeros((len(trajectories), horizon, num_acuity_scores))
rewards = np.zeros((len(trajectories), horizon))
actions = np.zeros((len(trajectories), horizon))
actionvecs = np.zeros((len(trajectories), horizon, num_actions))
subactions = np.zeros((len(trajectories), horizon, 2))
subactionvecs = np.zeros((len(trajectories), horizon, 5+5))

for i, traj_idx in tqdm(enumerate(trajectories), total=len(trajectories)):
    traj = df_data[df_data['traj'] == traj_idx].sort_values(by='step')
    traj_acuity = df_acuity[df_acuity['traj'] == traj_idx].sort_values(by='step')
    length = traj.shape[0]
    assert len(traj_acuity == length)
    
    icustayids[i] = traj[icustayid_col].iloc[0]
    lengths[i] = length
    times[i] = np.arange(horizon)
    times[i, length:] = 0
    notdones[i, :length-1] = 1
    observations[i, :length, :] = traj[obs_cols].values
    demographics[i, :length, :] = traj[dem_cols].values
    acuities[i, :length, :] = traj_acuity[acuity_cols].values
    rewards[i, :length] = traj[rew_col].values
    actions[i, :length] = traj['a:action'].values
    actionvecs[i, range(length), traj['a:action'].astype(int).values] = 1
    subactions[i, :length, :] = traj[['a:vaso', 'a:iv']].values
    subactionvecs[i, range(length), traj['a:vaso'].astype(int).values] = 1
    subactionvecs[i, range(length), 5+traj['a:iv'].astype(int).values] = 1

100%|██████████| 2757/2757 [00:04<00:00, 585.43it/s]


In [271]:
observations.shape

(2757, 11, 33)

In [272]:
# Eliminate single transition trajectories...
length_mask = (lengths > 1.0)
trajectories = trajectories[length_mask]
icustayids = icustayids[length_mask]
lengths = lengths[length_mask]
times = times[length_mask]
notdones = notdones[length_mask]
observations = observations[length_mask]
demographics = demographics[length_mask]
acuities = acuities[length_mask]
rewards = rewards[length_mask]
actions = actions[length_mask]
subactions = subactions[length_mask]
actionvecs = actionvecs[length_mask]
subactionvecs = subactionvecs[length_mask]

In [273]:
observations.shape

(2757, 11, 33)

In [274]:
torch.save({
    'index': torch.Tensor(trajectories).long(),
    'icustayids': torch.Tensor(icustayids).long(),
    'lengths': torch.Tensor(lengths).long(), 
    'times': torch.Tensor(times).float(), 
    'notdones': torch.Tensor(notdones).long(),
    'demographics': torch.Tensor(demographics).float(), 
    'observations': torch.Tensor(observations).float(), 
    'acuities': torch.Tensor(acuities).float(),
    'rewards': torch.Tensor(rewards).float(),
    'actions': torch.Tensor(actions).long(), 
    'actionvecs': torch.Tensor(actionvecs).float(), 
    'subactions': torch.Tensor(subactions).long(), 
    'subactionvecs': torch.Tensor(subactionvecs).float(), 
}, os.path.join(save_dir, val_file))

## Test DATA

In [275]:
df_data = test_data
df_acuity = test_acuity
trajectories = test_trajectories

In [276]:
icustayids = np.zeros((len(trajectories)), dtype=int)
lengths = np.zeros((len(trajectories)), dtype=int)
times = np.zeros((len(trajectories), horizon), dtype=int)
notdones = np.zeros((len(trajectories), horizon), dtype=int)
observations = np.zeros((len(trajectories), horizon, num_obs))
demographics = np.zeros((len(trajectories), horizon, num_dem))
acuities = np.zeros((len(trajectories), horizon, num_acuity_scores))
rewards = np.zeros((len(trajectories), horizon))
actions = np.zeros((len(trajectories), horizon))
actionvecs = np.zeros((len(trajectories), horizon, num_actions))
subactions = np.zeros((len(trajectories), horizon, 2))
subactionvecs = np.zeros((len(trajectories), horizon, 5+5))

for i, traj_idx in tqdm(enumerate(trajectories), total=len(trajectories)):
    traj = df_data[df_data['traj'] == traj_idx].sort_values(by='step')
    traj_acuity = df_acuity[df_acuity['traj'] == traj_idx].sort_values(by='step')
    length = traj.shape[0]
    assert len(traj_acuity == length)
    
    icustayids[i] = traj[icustayid_col].iloc[0]
    lengths[i] = length
    times[i] = np.arange(horizon)
    times[i, length:] = 0
    notdones[i, :length-1] = 1
    observations[i, :length, :] = traj[obs_cols].values
    demographics[i, :length, :] = traj[dem_cols].values
    acuities[i, :length, :] = traj_acuity[acuity_cols].values
    rewards[i, :length] = traj[rew_col].values
    actions[i, :length] = traj['a:action'].values
    actionvecs[i, range(length), traj['a:action'].astype(int).values] = 1
    subactions[i, :length, :] = traj[['a:vaso', 'a:iv']].values
    subactionvecs[i, range(length), traj['a:vaso'].astype(int).values] = 1
    subactionvecs[i, range(length), 5+traj['a:iv'].astype(int).values] = 1

100%|██████████| 2757/2757 [00:04<00:00, 594.38it/s]


In [277]:
observations.shape

(2757, 11, 33)

In [278]:
# Eliminate single transition trajectories...
length_mask = (lengths > 1.0)
trajectories = trajectories[length_mask]
icustayids = icustayids[length_mask]
lengths = lengths[length_mask]
times = times[length_mask]
notdones = notdones[length_mask]
observations = observations[length_mask]
demographics = demographics[length_mask]
acuities = acuities[length_mask]
rewards = rewards[length_mask]
actions = actions[length_mask]
subactions = subactions[length_mask]
actionvecs = actionvecs[length_mask]
subactionvecs = subactionvecs[length_mask]

In [279]:
observations.shape

(2757, 11, 33)

In [280]:
torch.save({
    'index': torch.Tensor(trajectories).long(),
    'icustayids': torch.Tensor(icustayids).long(),
    'lengths': torch.Tensor(lengths).long(), 
    'times': torch.Tensor(times).float(), 
    'notdones': torch.Tensor(notdones).long(),
    'demographics': torch.Tensor(demographics).float(), 
    'observations': torch.Tensor(observations).float(), 
    'acuities': torch.Tensor(acuities).float(),
    'rewards': torch.Tensor(rewards).float(),
    'actions': torch.Tensor(actions).long(), 
    'actionvecs': torch.Tensor(actionvecs).float(), 
    'subactions': torch.Tensor(subactions).long(), 
    'subactionvecs': torch.Tensor(subactionvecs).float(), 
}, os.path.join(save_dir, test_file))